#### Importing modules

In [23]:
import json 
import pandas as pd
import numpy as np

#### Read cowin csv file

In [24]:
data=pd.read_csv('cowin_vaccine_data_districtwise.csv')

#### Read census data xlsx file

In [25]:
cols=['State', 'District','Level',
       'Name', 'TRU','TOT_P', 'TOT_M', 'TOT_F']
census_data=pd.read_excel('DDW_PCA0000_2011_Indiastatedist.xlsx',usecols=cols)
#only considering total rather than rural and urban seperately for this question
census_data=census_data.loc[(census_data['TRU']=='Total') & (census_data['Level']=='DISTRICT')]
census_data['Name_Lower']=0
census_data['update_step']=np.nan
census_data.reset_index(inplace=True,drop=True)

#### Load modified json file from Q1

In [26]:
## district level
# district data from json
f=open('neighbor-districts-modified.json')
districts_data=json.load(f)
district_names=[]
district_ids=[]
for key in districts_data:
    district_names.append(key.split('/')[0])
    district_ids.append(key.split('/')[1])

/tmp/ipykernel_24351/2911079101.py:3: ResourceWarning: unclosed file <_io.TextIOWrapper name='neighbor-districts-modified.json' mode='r' encoding='UTF-8'>
  f=open('neighbor-districts-modified.json')


#### Editdist function for strings

In [27]:
def editdist(str1, str2): 
    m = len(str1)
    n = len(str2)
    # Create a table to store results of subproblems
    
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] 
  
    # Fill d[][] in bottom up manner 
    for i in range(m + 1): 
        for j in range(n + 1): 
  
            # If first string is empty, only option is to 
            # insert all characters of second string 
            if i == 0: 
                dp[i][j] = j    # Min. operations = j 
  
            # If second string is empty, only option is to 
            # remove all characters of second string 
            elif j == 0: 
                dp[i][j] = i    # Min. operations = i 
  
            # If last characters are same, ignore last char 
            # and recur for remaining string 
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # If last character are different, consider all 
            # possibilities and find minimum 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert 
                                   dp[i-1][j],        # Remove 
                                   dp[i-1][j-1])    # Replace 
    #print(dp)
    return dp[m][n] 

In [28]:
#dropping duplicate names from the cowin file
for i in range(len(district_ids)):
    dis=district_ids[i]
    if dis == 'MH_Aurangabad' or dis == 'CT_Bilaspur' or dis == 'UP_Hamirpur' or dis == 'CT_Balrampur' or dis == 'RJ_Pratapgarh':
        district_ids[i]='ItIs_ADuplicatedDistrict'

#### Getting common districts and updating census file district names

In [29]:
%%time
# get common districts with editdist
common_districts=[]
common_districts_list=[]
counter_list=np.zeros(len(district_ids))
for i in range(len(district_ids)):
    k=0
    a=district_ids[i].lower().split('_')
    for j in range(census_data.shape[0]):
        b=census_data['Name'][j].lower()
        if a[1].replace(' ','')==b.replace(' ','') and (district_ids[i] not in common_districts_list) and counter_list[i]==0:  
            counter_list[i]=1
            k=1
            common_districts.append(a[1])
            common_districts_list.append(district_ids[i])
            census_data.iloc[j,8]=district_ids[i]
            census_data.iloc[j,9]=1
            break
    if k==0:
        common_districts.append('#')
        common_districts_list.append('#')
            

CPU times: user 1.44 s, sys: 0 ns, total: 1.44 s
Wall time: 1.44 s


In [30]:
%%time
for j in range(census_data.shape[0]):
    b=census_data.Name_Lower[j]
    if b==0:
        b=census_data.Name[j].lower()
        for i in range(len(district_ids)):
            a=district_ids[i].lower().split('_')
            if (editdist(b.replace(' ',''),a[1].replace(' ',''))<2) and (b[0]==a[1][0]) and (counter_list[i]==0):
                common_districts[i]=a[1]
                common_districts_list[i]=district_ids[i]
                census_data.iloc[j,8]=district_ids[i]
                census_data.iloc[j,9]=2
                counter_list[i]==1
                break


CPU times: user 2.96 s, sys: 0 ns, total: 2.96 s
Wall time: 2.96 s


In [31]:
%%time
for j in range(census_data.shape[0]):
    b=census_data.Name_Lower[j]
    if b==0:
        b=census_data.Name[j].lower()
        for i in range(len(district_ids)):
            a=district_ids[i].lower().split('_')
            if (editdist(b.replace(' ',''),a[1].replace(' ',''))<3) and (b[0]==a[1][0])  and (counter_list[i]==0):
                common_districts[i]=a[1]
                common_districts_list[i]=district_ids[i]
                census_data.iloc[j,8]=district_ids[i]
                census_data.iloc[j,9]=3
                counter_list[i]=1
                break

CPU times: user 2.56 s, sys: 0 ns, total: 2.56 s
Wall time: 2.57 s


#### Prepare data frames for female and male vaccine numbers


In [32]:
# let's change those column names
#date==datetime.date(2021,8,14)
data_dose1=data.loc[:,(data.loc[0,]=='Female(Individuals Vaccinated)')].iloc[1:,:].fillna(0)
first_date_dose1=data_dose1.iloc[:,0]
data_dose1=data_dose1.astype(int).diff(axis=1)
data_dose1.iloc[:,0]=first_date_dose1
data_dose1['District']=data['District_Key']
data_dose2=data.loc[:,(data.loc[0,]=='Male(Individuals Vaccinated)')].iloc[1:,:].fillna(0)
first_date_dose2=data_dose2.iloc[:,0]
data_dose2=data_dose2.astype(int).diff(axis=1)
data_dose2.iloc[:,0]=first_date_dose2
data_dose2['District']=data['District_Key']
## Just keep in mind that dose1 represents female and other one male doses

#### To get asked dates binary list


In [33]:
import datetime
true_false_female=[]
for key in data_dose1.columns[:-1]:
    changed_key=key.split('.')[0].split('/')
    true_false_female.append(datetime.date(int(changed_key[2]),int(changed_key[1]),int(changed_key[0]))<datetime.date(2021,8,15))
    
true_false_female.append(False)

true_false_male=[]
for key in data_dose2.columns[:-1]:
    changed_key=key.split('.')[0].split('/')
    true_false_male.append(datetime.date(int(changed_key[2]),int(changed_key[1]),int(changed_key[0]))<datetime.date(2021,8,15))
    
true_false_male.append(False)

### Districts

#### Counts female and male doses for districts

In [34]:
district_female_male=[]
male_count=[]
female_count=[]
for i in range(len(common_districts_list)):
    a=common_districts_list[i]
    for j in range(data_dose1.shape[0]):
        if a==data_dose1['District'][j+1]:
            district_female_male.append(a)
            female_count.append(data_dose1.loc[data_dose1['District']==a, np.array(true_false_female)].astype(int).sum().sum())
            male_count.append(data_dose2.loc[data_dose2['District']==a, np.array(true_false_male)].astype(int).sum().sum())
            break

#### Counts female and male population for districts

In [35]:
total_male=[]
total_female=[]
district_male_female_pop=[]
for i in range(len(common_districts_list)):
    a=common_districts_list[i]
    for j in range(census_data.shape[0]):
        if a==census_data.iloc[j,8]:
            district_male_female_pop.append(a)
            total_female.append(census_data.iloc[j,7].astype(int))
            total_male.append(census_data.iloc[j,6].astype(int))
            break

#### Compute ratios

In [36]:
vaccinationratio=[i/j for i,j in zip(female_count,male_count)]
populationratio=[i/j for i,j in zip(total_female,total_male)]
ratioofratios=[i/j for i,j in zip(vaccinationratio,populationratio)]

#### Write in csv file

In [37]:
ratio_df=pd.DataFrame({ 'districtid':district_female_male, 'vaccinationratio':vaccinationratio, 'populationratio':populationratio, 'ratioofratios':ratioofratios})
ratio_df.sort_values('ratioofratios', axis = 0, ascending = True, kind='mergesort',inplace=True)
ratio_df.reset_index(inplace=True,drop=True)
ratio_df.to_csv('district-vaccination-population-ratio.csv',index=False)

### States

- [x] To avoid extra counting of population for districts that are not being considered, I have used districts df obtained above to work on states.




In [38]:
use_df=pd.DataFrame({'districtid':district_female_male,'female_dose':female_count,'male_dose':male_count,'female_pop':total_female,'male_pop':total_male})

In [39]:
state_list=[]
for i in range(len(district_female_male)):
    state_list.append(district_female_male[i].split('_')[0])
use_df['stateid']=state_list
unique_state=np.unique(np.array(use_df.stateid))

#### Counts female and male doses and population for states

In [40]:
state_female_dose=[]
state_male_dose=[]
state_female_pop=[]
state_male_pop=[]
for i in range(len(unique_state)):
    foo_df=use_df[use_df.stateid==unique_state[i]]
    state_female_dose.append(sum(foo_df.female_dose.tolist()))
    state_male_dose.append(sum(foo_df.male_dose.tolist()))
    state_female_pop.append(sum(foo_df.female_pop.tolist()))
    state_male_pop.append(sum(foo_df.male_pop.tolist()))

#### Compute ratios

In [41]:
vaccinationratio=[i/j for i,j in zip(state_female_dose,state_male_dose)]
populationratio=[i/j for i,j in zip(state_female_pop,state_male_pop)]
ratioofratios=[i/j for i,j in zip(vaccinationratio,populationratio)]

#### write in csvs

In [42]:
ratio_df_state=pd.DataFrame({ 'stateid':unique_state, 'vaccinationratio':vaccinationratio, 'populationratio':populationratio, 'ratioofratios':ratioofratios})
ratio_df_state.sort_values('ratioofratios', axis = 0, ascending = True, kind='mergesort',inplace=True)
ratio_df_state.reset_index(inplace=True,drop=True)
ratio_df_state.to_csv('state-vaccination-population-ratio.csv',index=False)

### Overall

In [43]:
vaccinationratio=sum(state_female_dose)/sum(state_male_dose)
populationratio=sum(state_female_pop)/sum(state_male_pop)
ratioofratios=vaccinationratio/populationratio

In [44]:
ratio_df_overall=pd.DataFrame({ 'overallid':['IN'], 'vaccinationratio':[vaccinationratio], 'populationratio':[populationratio], 'ratioofratios':[ratioofratios]})
ratio_df_overall.to_csv('overall-vaccination-population-ratio.csv',index=False)